In [1]:
import pandas as pd
import duckdb

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
data = {
    'name': ['Toufik', 'Jean-Nicolas', 'Daniel', 'Kaouter', 'Sylvie',
             'Sebastien', 'Diane', 'Romain', 'François', 'Anna',
             'Zeinaba', 'Gregory', 'Karima', 'Arthur', 'Benjamin'],
    'wage': [60000, 75000, 55000, 100000, 70000,
             90000, 65000, 100000, 68000, 85000,
             100000, 120000, 95000, 83000, 110000],
    'department': ['IT', 'HR', 'SALES', 'IT', 'IT',
                   'HR', 'SALES', 'IT', 'HR', 'SALES',
                   'IT', 'IT', 'HR', 'SALES', 'CEO'],
    'sex': ['H', 'H', 'H', 'F', 'F',
           'H', 'F', 'H', 'H', 'F',
           'F', 'H', 'F', 'H', 'H',]
}
wages = pd.DataFrame(data)
wages.head()

,name,wage,department,sex
0,Toufik,60000,IT,H
1,Jean-Nicolas,75000,HR,H
2,Daniel,55000,SALES,H
3,Kaouter,100000,IT,F
4,Sylvie,70000,IT,F


Obtenir l'index:

In [3]:
query = """
SELECT *,
ROW_NUMBER() OVER() AS index
FROM wages
ORDER BY department
"""
duckdb.sql(query).df().head(10)

,name,wage,department,sex,index
0,Benjamin,110000,CEO,H,15
1,Jean-Nicolas,75000,HR,H,2
2,Sebastien,90000,HR,H,6
3,François,68000,HR,H,9
4,Karima,95000,HR,F,13
5,Toufik,60000,IT,H,1
6,Kaouter,100000,IT,F,4
7,Sylvie,70000,IT,F,5
8,Romain,100000,IT,H,8
9,Zeinaba,100000,IT,F,11


Le numéro de ligne par département:

In [4]:
query = """
SELECT *,
ROW_NUMBER() OVER(PARTITION BY department) AS index
FROM wages
"""
duckdb.sql(query).df().head(10)

,name,wage,department,sex,index
0,Toufik,60000,IT,H,1
1,Kaouter,100000,IT,F,2
2,Sylvie,70000,IT,F,3
3,Romain,100000,IT,H,4
4,Zeinaba,100000,IT,F,5
5,Gregory,120000,IT,H,6
6,Benjamin,110000,CEO,H,1
7,Daniel,55000,SALES,H,1
8,Diane,65000,SALES,F,2
9,Anna,85000,SALES,F,3


Exercice: obtenez le classement entre les hommes et les femmes

In [5]:
query = """
SELECT *,
ROW_NUMBER() OVER(PARTITION BY sex) AS index
FROM wages

"""
duckdb.sql(query).df()

,name,wage,department,sex,index
0,Toufik,60000,IT,H,1
1,Jean-Nicolas,75000,HR,H,2
2,Daniel,55000,SALES,H,3
3,Sebastien,90000,HR,H,4
4,Romain,100000,IT,H,5
5,François,68000,HR,H,6
6,Gregory,120000,IT,H,7
7,Arthur,83000,SALES,H,8
8,Benjamin,110000,CEO,H,9
9,Kaouter,100000,IT,F,1


In [6]:
# %load solutions/14row_number_by_sex.py
query = """
SELECT *,
ROW_NUMBER() OVER(
    PARTITION BY sex
    ) AS index
FROM wages
"""
duckdb.sql(query).df().head(11)


,name,wage,department,sex,index
0,Toufik,60000,IT,H,1
1,Jean-Nicolas,75000,HR,H,2
2,Daniel,55000,SALES,H,3
3,Sebastien,90000,HR,H,4
4,Romain,100000,IT,H,5
5,François,68000,HR,H,6
6,Gregory,120000,IT,H,7
7,Arthur,83000,SALES,H,8
8,Benjamin,110000,CEO,H,9
9,Kaouter,100000,IT,F,1


Le numéro de ligne par département, du plus gros au plus petit salaire:

In [7]:
query = """
SELECT *,
ROW_NUMBER() OVER(PARTITION BY department ORDER BY wage DESC) AS index
FROM wages

"""
duckdb.sql(query).df()

,name,wage,department,sex,index
0,Gregory,120000,IT,H,1
1,Kaouter,100000,IT,F,2
2,Romain,100000,IT,H,3
3,Zeinaba,100000,IT,F,4
4,Sylvie,70000,IT,F,5
5,Toufik,60000,IT,H,6
6,Karima,95000,HR,F,1
7,Sebastien,90000,HR,H,2
8,Jean-Nicolas,75000,HR,H,3
9,François,68000,HR,H,4


In [8]:
# %load solutions/15row_number_by_dpt_order_by_wage.py
query = """
SELECT *,
ROW_NUMBER() OVER(
    PARTITION BY department
    ORDER BY wage DESC
    ) AS index
FROM wages
LIMIT 10
"""
duckdb.sql(query).df()


,name,wage,department,sex,index
0,Gregory,120000,IT,H,1
1,Kaouter,100000,IT,F,2
2,Romain,100000,IT,H,3
3,Zeinaba,100000,IT,F,4
4,Sylvie,70000,IT,F,5
5,Toufik,60000,IT,H,6
6,Karima,95000,HR,F,1
7,Sebastien,90000,HR,H,2
8,Jean-Nicolas,75000,HR,H,3
9,François,68000,HR,H,4


Ici, on peut s'apercevoir que Kaouter, Romain et Zeinaba ont le même salaire. <br />
pourtant leur "row_number" continue de progresser.

Clairement, ce n'est pas la fonction idéale pour gérer les classements...

Exercice avant de passer à la suite: obtenez le classement des salaires par sexe:

In [10]:
query = """
SELECT *,
ROW_NUMBER() OVER(PARTITION BY sex ORDER BY wage DESC) AS index
FROM wages

"""
duckdb.sql(query).df()

,name,wage,department,sex,index
0,Gregory,120000,IT,H,1
1,Benjamin,110000,CEO,H,2
2,Romain,100000,IT,H,3
3,Sebastien,90000,HR,H,4
4,Arthur,83000,SALES,H,5
5,Jean-Nicolas,75000,HR,H,6
6,François,68000,HR,H,7
7,Toufik,60000,IT,H,8
8,Daniel,55000,SALES,H,9
9,Kaouter,100000,IT,F,1


In [11]:
# %load solutions/16row_number_by_sex_order_by_wage.py
query = """
SELECT *,
ROW_NUMBER() OVER(
    PARTITION BY sex
    ORDER BY wage DESC) 
AS index
FROM wages
LIMIT 10 
"""
duckdb.sql(query).df().head(11)


,name,wage,department,sex,index
0,Gregory,120000,IT,H,1
1,Benjamin,110000,CEO,H,2
2,Romain,100000,IT,H,3
3,Sebastien,90000,HR,H,4
4,Arthur,83000,SALES,H,5
5,Jean-Nicolas,75000,HR,H,6
6,François,68000,HR,H,7
7,Toufik,60000,IT,H,8
8,Daniel,55000,SALES,H,9
9,Kaouter,100000,IT,F,1


### Comment gérer les égalités ? RANK vs. DENSE_RANK

In [12]:
query = """
SELECT *,
RANK() OVER(
    PARTITION BY department
    ORDER BY wage DESC) AS index
FROM wages
"""
duckdb.sql(query).df().head(10)

,name,wage,department,sex,index
0,Gregory,120000,IT,H,1
1,Kaouter,100000,IT,F,2
2,Romain,100000,IT,H,2
3,Zeinaba,100000,IT,F,2
4,Sylvie,70000,IT,F,5
5,Toufik,60000,IT,H,6
6,Karima,95000,HR,F,1
7,Sebastien,90000,HR,H,2
8,Jean-Nicolas,75000,HR,H,3
9,François,68000,HR,H,4


A comparer avec dense_rank :

In [14]:
query = """
SELECT *,
DENSE_RANK() OVER(
    PARTITION BY department 
    ORDER BY wage DESC) AS index
FROM wages
"""
duckdb.sql(query).df().head(10)

,name,wage,department,sex,index
0,Benjamin,110000,CEO,H,1
1,Gregory,120000,IT,H,1
2,Kaouter,100000,IT,F,2
3,Romain,100000,IT,H,2
4,Zeinaba,100000,IT,F,2
5,Sylvie,70000,IT,F,3
6,Toufik,60000,IT,H,4
7,Anna,85000,SALES,F,1
8,Arthur,83000,SALES,H,2
9,Diane,65000,SALES,F,3


Avez-vous trouvé la différence ?

Elle ne se situe pas au niveau du ranking des valeurs qui sont à égalité <br />
La différence se situe au niveau des valeurs suivantes.

- avec RANK, on obtient 1, 2, 2, 2, 5(!), 6(!): le classement reprends comme si les égalités avaient eu le rang 2, 3 ,4
- avec DENSE_RANK, on obtient 1, 2, 2, 2, 3, 4: le classement reprends immédiatement après la valeur de l'égalité

Personnellement, je trouve DENSE_RANK plus intuitif <br /> et je n'ai encore jamais rencontré de cas de figure où j'ai eu besoin du RANK

<img src="images/rank_useless_staff_engineer_apple.png" />

## Exercices

Obtenez le classement des salaires par sexe <br />
(S'il y a une égalité pour la première place, <br />
on souhaite que le 3ème ait le rang "2")

In [17]:
query = """
SELECT *,
DENSE_RANK() OVER(PARTITION BY sex ORDER BY wage DESC) AS index
FROM wages

"""
duckdb.sql(query).df()

,name,wage,department,sex,index
0,Kaouter,100000,IT,F,1
1,Zeinaba,100000,IT,F,1
2,Karima,95000,HR,F,2
3,Anna,85000,SALES,F,3
4,Sylvie,70000,IT,F,4
5,Diane,65000,SALES,F,5
6,Gregory,120000,IT,H,1
7,Benjamin,110000,CEO,H,2
8,Romain,100000,IT,H,3
9,Sebastien,90000,HR,H,4


In [19]:
# %load solutions/17first_ranking_exercice.py
query = """
SELECT *,
DENSE_RANK() OVER(
    PARTITION BY sex 
    ORDER BY wage DESC) AS index
FROM wages
"""
duckdb.sql(query).df().head(13)


,name,wage,department,sex,index
0,Gregory,120000,IT,H,1
1,Benjamin,110000,CEO,H,2
2,Romain,100000,IT,H,3
3,Sebastien,90000,HR,H,4
4,Arthur,83000,SALES,H,5
5,Jean-Nicolas,75000,HR,H,6
6,François,68000,HR,H,7
7,Toufik,60000,IT,H,8
8,Daniel,55000,SALES,H,9
9,Kaouter,100000,IT,F,1


### Cas pratique capteur retail

On souhaite savoir quelle entrée de l'hypermarché <br />
est la plus fréquentée les Samedi

In [20]:
import random
random.seed(42)

In [21]:
df = pd.read_csv("data/capteur_a_retrail.csv")
df_porte_b = df.copy()
# On ajoute une porte_b
df_porte_b["capteur_id"] = "porte_b"
df_porte_b["visiteurs_count"] = df_porte_b["visiteurs_count"].apply(lambda x: round(x * (random.random() + 0.5), 0))
df = pd.concat([df, df_porte_b])
df.sample(7)

,date,capteur_id,visiteurs_count,weekday,moyenne_du_mois,threshold_twenty_pct
22,2023-08-26,porte_a,6000.0,7,4920.0,3936.0
28,2023-09-02,porte_b,5390.0,7,4700.0,3760.0
19,2023-08-23,porte_b,2786.0,4,4920.0,3936.0
11,2023-08-14,porte_a,3900.0,2,4920.0,3936.0
4,2023-08-05,porte_b,7419.0,7,4920.0,3936.0
15,2023-08-18,porte_b,5225.0,6,4920.0,3936.0
2,2023-08-03,porte_b,3410.0,5,4920.0,3936.0


Exercice: faites une requête pour avoir la moyenne glissante des visiteurs <br />
les Samedi, pour le capteur porte_a et le capteur porte_b

In [24]:
query = """
SELECT 
    *,
    AVG(visiteurs_count) OVER(PARTITION BY capteur_id ORDER BY date) AS avg_visitors_weekday
FROM df
WHERE weekday = 7

"""
duckdb.sql(query).df()

,date,capteur_id,visiteurs_count,weekday,moyenne_du_mois,threshold_twenty_pct,avg_visitors_weekday
0,2023-08-05,porte_a,6000.0,7,4920.0,3936.0,6000.000000
1,2023-08-12,porte_a,6000.0,7,4920.0,3936.0,6000.000000
2,2023-08-19,porte_a,5500.0,7,4920.0,3936.0,5833.333333
3,2023-08-26,porte_a,6000.0,7,4920.0,3936.0,5875.000000
4,2023-09-02,porte_a,4000.0,7,4700.0,3760.0,5500.000000
5,2023-08-05,porte_b,7419.0,7,4920.0,3936.0,7419.000000
6,2023-08-12,porte_b,4312.0,7,4920.0,3936.0,5865.500000
7,2023-08-19,porte_b,3962.0,7,4920.0,3936.0,5231.000000
8,2023-08-26,porte_b,5042.0,7,4920.0,3936.0,5183.750000
9,2023-09-02,porte_b,5390.0,7,4700.0,3760.0,5225.000000


In [26]:
# %load solutions/18avg_visitors_count_by_capteur_id.py
query = """
SELECT *,
AVG(visiteurs_count) OVER(
    PARTITION BY capteur_id 
    ORDER BY Date
    ) AS avg_visitors_weekday
FROM df
WHERE weekday = 7
ORDER BY capteur_id, date
"""
duckdb.sql(query).df()


,date,capteur_id,visiteurs_count,weekday,moyenne_du_mois,threshold_twenty_pct,avg_visitors_weekday
0,2023-08-05,porte_a,6000.0,7,4920.0,3936.0,6000.000000
1,2023-08-12,porte_a,6000.0,7,4920.0,3936.0,6000.000000
2,2023-08-19,porte_a,5500.0,7,4920.0,3936.0,5833.333333
3,2023-08-26,porte_a,6000.0,7,4920.0,3936.0,5875.000000
4,2023-09-02,porte_a,4000.0,7,4700.0,3760.0,5500.000000
5,2023-08-05,porte_b,7419.0,7,4920.0,3936.0,7419.000000
6,2023-08-12,porte_b,4312.0,7,4920.0,3936.0,5865.500000
7,2023-08-19,porte_b,3962.0,7,4920.0,3936.0,5231.000000
8,2023-08-26,porte_b,5042.0,7,4920.0,3936.0,5183.750000
9,2023-09-02,porte_b,5390.0,7,4700.0,3760.0,5225.000000


Attention, cette requête fonctionne parce que le fenêtrage se fait sur la donnée restante <b> après la clause WHERE </b>

Si on enlève le WHERE weekday = 7, les calculs ne sont plus bons

In [28]:
# CETTE REQUETE N'EST PAS BONNE !!

query = """
SELECT *,
AVG(visiteurs_count) OVER(
    PARTITION BY capteur_id 
    ORDER BY Date
    ) AS avg_visitors_weekday
FROM df
ORDER BY capteur_id, weekday, date
"""
duckdb.sql(query).df().query("weekday == 7")


,date,capteur_id,visiteurs_count,weekday,moyenne_du_mois,threshold_twenty_pct,avg_visitors_weekday
25,2023-08-05,porte_a,6000.0,7,4920.0,3936.0,5080.000000
26,2023-08-12,porte_a,6000.0,7,4920.0,3936.0,5045.454545
27,2023-08-19,porte_a,5500.0,7,4920.0,3936.0,4847.058824
28,2023-08-26,porte_a,6000.0,7,4920.0,3936.0,4939.130435
29,2023-09-02,porte_a,4000.0,7,4700.0,3760.0,4912.068966
55,2023-08-05,porte_b,7419.0,7,4920.0,3936.0,4475.200000
56,2023-08-12,porte_b,4312.0,7,4920.0,3936.0,4386.727273
57,2023-08-19,porte_b,3962.0,7,4920.0,3936.0,4176.352941
58,2023-08-26,porte_b,5042.0,7,4920.0,3936.0,4478.782609
59,2023-09-02,porte_b,5390.0,7,4700.0,3760.0,4443.862069


Pour avoir les bons résultats sur l'ensemble de la donnée, <br />
il faut préciser davantage le fenêtrage en partitionnant par capteur_id ET par weekday:

In [30]:
query = """
SELECT
    *,
    AVG(visiteurs_count) OVER(PARTITION BY capteur_id, weekday ORDER BY DATE) AS avg_visitors_weekday
FROM df
ORDER BY capteur_id, weekday, date
"""
duckdb.sql(query).df()

,date,capteur_id,visiteurs_count,weekday,moyenne_du_mois,threshold_twenty_pct,avg_visitors_weekday
0,2023-08-07,porte_a,4200.0,2,4920.0,3936.0,4200.000000
1,2023-08-14,porte_a,3900.0,2,4920.0,3936.0,4050.000000
2,2023-08-21,porte_a,4300.0,2,4920.0,3936.0,4133.333333
3,2023-08-28,porte_a,4300.0,2,4920.0,3936.0,4175.000000
4,2023-09-04,porte_a,4500.0,2,4700.0,3760.0,4240.000000
5,2023-08-01,porte_a,4200.0,3,4920.0,3936.0,4200.000000
6,2023-08-08,porte_a,4700.0,3,4920.0,3936.0,4450.000000
7,2023-08-15,porte_a,4200.0,3,4920.0,3936.0,4366.666667
8,2023-08-22,porte_a,4900.0,3,4920.0,3936.0,4500.000000
9,2023-08-29,porte_a,4750.0,3,4920.0,3936.0,4550.000000


In [31]:
# %load solutions/19correct_avg_visitors_count_by_capteur_id.py
query = """
SELECT *,
AVG(visiteurs_count) OVER(
    PARTITION BY capteur_id, weekday
    ORDER BY Date
    ) AS avg_visitors_weekday
FROM df
ORDER BY capteur_id, weekday, date
"""
duckdb.sql(query).df().query("weekday == 7")



,date,capteur_id,visiteurs_count,weekday,moyenne_du_mois,threshold_twenty_pct,avg_visitors_weekday
25,2023-08-05,porte_a,6000.0,7,4920.0,3936.0,6000.000000
26,2023-08-12,porte_a,6000.0,7,4920.0,3936.0,6000.000000
27,2023-08-19,porte_a,5500.0,7,4920.0,3936.0,5833.333333
28,2023-08-26,porte_a,6000.0,7,4920.0,3936.0,5875.000000
29,2023-09-02,porte_a,4000.0,7,4700.0,3760.0,5500.000000
55,2023-08-05,porte_b,7419.0,7,4920.0,3936.0,7419.000000
56,2023-08-12,porte_b,4312.0,7,4920.0,3936.0,5865.500000
57,2023-08-19,porte_b,3962.0,7,4920.0,3936.0,5231.000000
58,2023-08-26,porte_b,5042.0,7,4920.0,3936.0,5183.750000
59,2023-09-02,porte_b,5390.0,7,4700.0,3760.0,5225.000000


Utilisez cette requête pour obtenir, chaque semaine, le classement entre les capteurs_a et _b <br />
(Hint: utilisez une CTE)

In [48]:
query = """
WITH moving_avg_visitors AS (
SELECT
    *,
    AVG(visiteurs_count) OVER(
        PARTITION BY capteur_id, weekday ORDER BY date) AS avg_visitors_weekday
FROM df
ORDER BY capteur_id, weekday
)

SELECT
    *,
    DENSE_RANK() OVER(PARTITION BY date ORDER BY avg_visitors_weekday DESC) AS updated_ranking
FROM moving_avg_visitors
ORDER BY updated_ranking, date

"""
duckdb.sql(query).df()

,date,capteur_id,visiteurs_count,weekday,moyenne_du_mois,threshold_twenty_pct,avg_visitors_weekday,updated_ranking
0,2023-08-01,porte_b,4786.0,3,4920.0,3936.0,4786.000000,1
1,2023-08-02,porte_a,5300.0,4,4920.0,3936.0,5300.000000,1
2,2023-08-03,porte_a,4400.0,5,4920.0,3936.0,4400.000000,1
3,2023-08-04,porte_a,5500.0,6,4920.0,3936.0,5500.000000,1
4,2023-08-05,porte_b,7419.0,7,4920.0,3936.0,7419.000000,1
5,2023-08-07,porte_b,4942.0,2,4920.0,3936.0,4942.000000,1
6,2023-08-08,porte_b,6543.0,3,4920.0,3936.0,5664.500000,1
7,2023-08-09,porte_a,5300.0,4,4920.0,3936.0,5300.000000,1
8,2023-08-10,porte_a,4400.0,5,4920.0,3936.0,4400.000000,1
9,2023-08-11,porte_a,5500.0,6,4920.0,3936.0,5500.000000,1


In [46]:
# %load solutions/20updated_ranking.py
query = """
WITH moving_avg_visitors AS (
    SELECT *,
    AVG(visiteurs_count) OVER(
        PARTITION BY capteur_id, weekday
        ORDER BY Date
        ) AS avg_visitors_weekday,
    FROM df
    ORDER BY capteur_id, weekday, date
)

SELECT * , 
DENSE_RANK() OVER(
    PARTITION BY date
    ORDER BY avg_visitors_weekday DESC
) AS updated_ranking 
FROM 
moving_avg_visitors
ORDER BY updated_ranking, date
"""
duckdb.sql(query).df().query("weekday == 7")


,date,capteur_id,visiteurs_count,weekday,moyenne_du_mois,threshold_twenty_pct,avg_visitors_weekday,updated_ranking
4,2023-08-05,porte_b,7419.0,7,4920.0,3936.0,7419.000000,1
10,2023-08-12,porte_a,6000.0,7,4920.0,3936.0,6000.000000,1
16,2023-08-19,porte_a,5500.0,7,4920.0,3936.0,5833.333333,1
22,2023-08-26,porte_a,6000.0,7,4920.0,3936.0,5875.000000,1
28,2023-09-02,porte_a,4000.0,7,4700.0,3760.0,5500.000000,1
34,2023-08-05,porte_a,6000.0,7,4920.0,3936.0,6000.000000,2
40,2023-08-12,porte_b,4312.0,7,4920.0,3936.0,5865.500000,2
46,2023-08-19,porte_b,3962.0,7,4920.0,3936.0,5231.000000,2
52,2023-08-26,porte_b,5042.0,7,4920.0,3936.0,5183.750000,2
58,2023-09-02,porte_b,5390.0,7,4700.0,3760.0,5225.000000,2
